# Lesson 1: Your first agent with Amazon Bedrock

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


## Start of the lesson

In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': 'a9f6042c-4a42-4936-bb52-948930c4495f',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 19 Oct 2024 03:37:15 GMT',
   'content-type': 'application/json',
   'content-length': '562',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a9f6042c-4a42-4936-bb52-948930c4495f',
   'x-amz-apigw-id': 'f4No1GS1PHcEmHg=',
   'x-amzn-trace-id': 'Root=1-6713296b-34059ae713f1a2af384c8f90'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:471112677410:agent/UDMKFTEARV',
  'agentId': 'UDMKFTEARV',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::471112677410:role/c135316a3428926l8020517t1w47111267-BedrockAgentRole-h0fTp4prg5Mi',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 10, 19, 3, 37, 15, 519230, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': 'de82dce5-72b9-4ac2-9e81-4d743f6a7c9b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 19 Oct 2024 03:37:19 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'de82dce5-72b9-4ac2-9e81-4d743f6a7c9b',
   'x-amz-apigw-id': 'f4NphHmqPHcEAqg=',
   'x-amzn-trace-id': 'Root=1-6713296f-5df4bbe54f482feb33589546'},
  'RetryAttempts': 0},
 'agentId': 'UDMKFTEARV',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 10, 19, 3, 37, 19, 919827, tzinfo=tzlocal())}

In [ ]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED


In [ ]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

In [ ]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [ ]:
import uuid

In [ ]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [ ]:
event_stream = invoke_agent_response["completion"]

In [ ]:
for event in event_stream:
    print(event)

In [ ]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [ ]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)